- **Project Title:** *Explainable Recommendations with BigQuery AI: From Profiles to Insights*
- **Problem Statement:** Most recommendation systems function as “black boxes,” surfacing items without offering context or transparency. This project uses BigQuery AI to automatically generate structured profiles of users and items, transform them into embeddings, and apply vector search to retrieve the most relevant matches. Each recommendation is then paired with an LLM-generated explanation, providing narrative reasoning that makes the system’s suggestions interpretable and trustworthy.
- **Impact Statement:** This solution shows how organizations can build scalable, explainable recommendation engines directly within BigQuery without the need for specialized machine learning teams. By combining personalization with transparency, the approach improves user trust, helps businesses uncover actionable behavioral patterns, and extends seamlessly to domains such as e-commerce and media.
- **Project Highlights**
    - Developed adaptable, structured profiles for users and recipes using BigQuery AI, enabling easy customization for various business domains.
    - Demonstrated that a straightforward recommendation system powered by BigQuery AI can slightly outperform the ALS baseline, simplifying implementation and highlighting the value of LLMs in recommendation workflows.
    - Created an interactive UI for users to browse personalized recommendations alongside clear, LLM-generated explanations.

# User & Recipe Profiles with BigQuery AI: From Embeddings to "Explainable" Recommendations


Inspired by the DoorDash blog post on [Profile Generation with LLMs](https://careersatdoordash.com/blog/doordash-profile-generation-llms-understanding-consumers-merchants-and-items/), this notebook showcases how to build user and recipe profiles leveraging BigQuery AI. It also outlines a workflow for generating potential recommendations, each paired with an "LLM-generated hypothesis" to provide narrative context for the suggestions.

This project addresses the challenges of **Approach 1 (The AI Architect)** and **Approach 2 (The Semantic Detective)** as described in the [Kaggle BigQuery Hackathon 2025](https://www.kaggle.com/competitions/bigquery-ai-hackathon/overview).

This notebook is structured as follows:

1. Data Sources
    - Datasets used (including sources and descriptions)
    - Preprocessing, splitting and uploading to BigQuery

2. (Approach 1) Recipe Profiles Generation
    - Profile schema design & prompt engineering
    - Using BigQuery AI to generate recipe profiles

3. (Approach 1) User Profiles Generation
    - Profile schema design & prompt engineering
    - Using BigQuery AI to generate user profiles

4. (Approach 2) Vector Search vs ALS
    - Using BigQuery AI to generate text embeddings
    - Simple collaborative filtering
    - Vector search for recommendations
        - Naive approach
        - HyDE (Hypothetical Document Embedding) approach
    - Comparing results with ALS baseline

5. (Approach 1) Explanation Generation
    - Prompt engineering for explanation generation
    - Using BigQuery AI to generate explanations for recommendations

6. (Approach 2) UI with Streamlit 
    - Hands-on interface to browse recommendations and their rationales
        - Automatic Personalized Recommendations
        - Semantic Search of Recipes

7. Summary and Conclusions
    - Feedback on BigQuery AI features
    - User Survey on BigQuery AI features

## Setup & Project Creation

In [ ]:
import ast
import seaborn as sns
from tqdm import tqdm
import os, json
os.chdir(os.path.dirname(os.path.abspath(__file__)))
import matplotlib.pyplot as plt
from IPython.display import display
from pydantic import BaseModel, Field
from typing import List, Dict, Optional
import os, ast, numpy as np, pandas as pd
import random
from scipy import sparse
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from dotenv import load_dotenv
import subprocess
import bigframes.pandas as bpd
from google.cloud import bigquery

np.set_printoptions(suppress=True, precision=2)
pd.set_option("display.float_format", "{:.3f}".format)

load_dotenv()

In [ ]:
os.environ['PROJECT_ID'] = input("Enter your Google Cloud Project ID: ")
PROJECT_ID = os.environ['PROJECT_ID']

subprocess.run(['gcloud', 'auth', 'login'])
subprocess.run(['gcloud', 'config', 'set', 'project', PROJECT_ID])
subprocess.run(['gcloud', 'auth', 'application-default', 'set-quota-project', PROJECT_ID])

bpd.options.bigquery.project = PROJECT_ID

random.seed(0)

CONNECTION_ID = 'us.kaggle-connection'
SCHEMA_NAME = 'foodrecsys'
VALID_INTERACTIONS = f"{PROJECT_ID}.{SCHEMA_NAME}.valid_interactions_windowed"
TRAIN_INTERACTIONS = f"{PROJECT_ID}.{SCHEMA_NAME}.train_interactions_windowed"
SUBSET_RECIPE_IDS = f"{PROJECT_ID}.{SCHEMA_NAME}.final_recipes"
SUBSET_USERS_IDS = f"{PROJECT_ID}.{SCHEMA_NAME}.final_users"

RECIPES_ALL = f"{PROJECT_ID}.{SCHEMA_NAME}.recipes"
OUT_DIM = 1024

RECIPES_PARSED = f'{SCHEMA_NAME}.recipes_parsed'
RECIPES_PROFILES_TABLE = f"{SCHEMA_NAME}.recipe_profiles"

USERS_PARSED = f'{SCHEMA_NAME}.users_parsed'
USERS_PROFILES_TABLE = f"{SCHEMA_NAME}.user_profiles"

VECTOR_SEARCH_RESULTS_TABLE = f"{SCHEMA_NAME}.vector_search_results"

client = bigquery.Client()

In [ ]:
def schema_to_prompt_with_descriptions(model_class) -> str:
    prompt = ""
    for k, v in model_class.model_json_schema()['properties'].items():
        desc = v.get('description', '')
        prompt += f" {k} ({desc}) "
    return f"[ {prompt} ]"

Following the tutorials on [GitHub](https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/use-cases/applying-llms-to-data/bigquery_embeddings_vector_search.ipynb) we create a remote connection to the Text Embedding model

In [ ]:
client.query_and_wait(f"""
CREATE OR REPLACE MODEL `{SCHEMA_NAME}.text_embedding_model`
REMOTE WITH 
    CONNECTION `{CONNECTION_ID}`
    OPTIONS (ENDPOINT = 'gemini-embedding-001');
""")

# Data Sources

For the dataset selection, three considerations were made:
- The datasets are publicly available (in line with the competition guidelines)
- The dataset should align with the goal of addressing a real-world problem using solutions that integrate naturally with SQL workflows.
- The dataset needs to include unstructured, messy, and mixed data


We utilize the [FoodRecSysV1](https://www.kaggle.com/datasets/elisaxxygao/foodrecsysv1) dataset from Kaggle, which provides comprehensive user interactions with recipes; including ratings, comments, and rich recipe metadata. This dataset closely mirrors the real-world challenges encountered by platforms such as Cookpad, DoorDash, UberEats, and Rappi.

Other datasets like [MealRec](https://arxiv.org/abs/2205.12133) and [MealRec++](https://github.com/WUT-IDEA/MealRecPlus) were also considered (and we can apply the same methodology of this notebook), but for the sake of simplicity and ease to start, we opted for FoodRecSysV1.


In [ ]:

# TODO: Read the dataset and reproduce the results into a new BigQuery dataset

We'll finally use a total of ...

# Recipe Profiles Generation

We start cleaning and parsing some columns to make them suitable and more understandable for the LLM.

Following the [DoorDash blog](https://careersatdoordash.com/blog/doordash-profile-generation-llms-understanding-consumers-merchants-and-items/) we define the structure and attributes of the "Recipe Profile" (item profiles). In this case we use a Pydantic model to define the schema, which then parsed as part of the prompt using the function `schema_to_prompt_with_descriptions`.

In [ ]:
class RecipeProfile(BaseModel):
    food_type: str = Field(description="Type of food, e.g., dessert, main course, appetizer")
    cuisine_type: str = Field(description="Cuisine type, e.g., Italian, Chinese, Mexican, American")
    dietary_preferences: List[str] = Field(description="Dietary preferences, e.g., omnivore, vegetarian, vegan, gluten-free")
    flavor_profile: List[str] = Field(description="Flavor profile, e.g., spicy, sweet, savory")
    serving_daypart: List[str] = Field(description="Suitable dayparts, e.g., breakfast, lunch, dinner")
    notes: str = Field(description="Short rationale summarizing the recipe profile")
    target_audience: str = Field(description="Types of users who would likely enjoy this recipe based on cooking skill level, flavor intensity, dietary needs, and lifestyle preferences. Helps recommendation systems match recipes to appropriate user profiles.")
    justification: str = Field(description="Detailed explanation of how the profile was determined Describe why the food type, cuisine type, dietary preferences, flavor profile, and serving daypart were chosen based on the ingredients and cooking directions. Is not allowed to use quotes or complex punctuation in this field.")


This schema could be enhanced by using more restrictive field types, such as `Enums` or `Literal` for categorical attributes. This would make it easier to apply business logic based on the extracted category metadata generated by the LLM in future iterations. 

Then, we define the prompt template to generate the recipe profiles, as follow:

In [ ]:
recipe_profile_prompt = f"""Based on the title, ingredients, cooking directions and percent daily values provided, create a recipe profile that summarizes the key characteristics of this recipe. Your response must follow this exact structure: {schema_to_prompt_with_descriptions(RecipeProfile)}. IMPORTANT: Do not use quotation marks or complex punctuation in your response. Use simple words and avoid any quotes, apostrophes, or special characters."""


In [ ]:
recipe_profile_generation_query = f"""
WITH ai_responses AS (
  SELECT 
    s.recipe_id, 
    s.title, 
    s.ingredients, 
    s.cooking_directions, 
    s.nutritions, 
    s.reviews, 
    s.parsed_ingredients, 
    s.parsed_recipe,
    AI.GENERATE(('{recipe_profile_prompt}', s.parsed_ingredients, s.parsed_recipe, s.percent_daily_values),
        connection_id => '{CONNECTION_ID}',
        endpoint => 'gemini-2.5-flash',
        model_params => JSON '{{"generationConfig":{{"temperature": 1.0, "maxOutputTokens": 2048, "thinking_config": {{"thinking_budget": 1024}} }} }}',
        output_schema => 'food_type STRING, cuisine_type STRING, dietary_preferences ARRAY<STRING>, flavor_profile ARRAY<STRING>, serving_daypart ARRAY<STRING>, notes STRING, target_audience STRING, justification STRING'
    ) AS ai_result
  FROM (SELECT * FROM `{RECIPES_PARSED}`) s
)
SELECT 
  *,
  ai_result.full_response AS recipe_profile,
  JSON_EXTRACT_SCALAR(ai_result.full_response, '$.candidates[0].content.parts[0].text') AS recipe_profile_text
FROM ai_responses
"""

# User Profiles Generation

For the user profile generation, we take advantage of the context window size of the Gemini models to provide a list of their historical interactions (ratings and comments) as part of the prompt.

In [ ]:
def get_user_history(user_id: int, n: int = 25, k_min: int = 5) -> list:
    pass

def format_user_history(user_history: list[dict]) -> str:
    pass

As in the recipe profiles, we define a Pydantic model, that it can be improved and adapted to the specific business case.

In [ ]:
class UserProfile(BaseModel):
    liked_cuisines: List[str] = Field(description="List of cuisines the user enjoys most, ranked by preference based on their interaction history and ratings")
    cuisine_preference: str = Field(description="Primary cuisine type the user gravitates towards (e.g., Mediterranean, Asian Fusion, Traditional American)")
    dietary_preference: str = Field(description="Main dietary restriction or lifestyle the user follows (e.g., Vegetarian, Low-carb, No restrictions)")

    food_preferences: List[str] = Field(description="Preferred food categories and meal types (e.g., comfort food, healthy salads, baked goods, grilled meats)")
    top_cuisine_choices: List[str] = Field(description="Specific regional or ethnic cuisines the user frequently rates highly (e.g., Thai, Southern BBQ, French pastry)")
    dietary_preferences: List[str] = Field(description="Dietary restrictions, health considerations, or eating patterns (e.g., gluten-free, plant-based, high-protein, dairy-free)")
    flavor_preferences: List[str] = Field(description="Dominant taste profiles and flavor characteristics the user seeks (e.g., bold and spicy, mild and creamy, tangy and citrusy)")
    daypart_preferences: List[str] = Field(description="Preferred times of day for different meal types based on rating patterns (e.g., hearty breakfast, light lunch, elaborate dinner)")
    lifestyle_tags: List[str] = Field(description="Behavioral patterns and cooking style indicators inferred from recipe choices (e.g., quick meals, entertainer, health-conscious, experimental cook)")
    convenience_preference: str = Field(description="Preference for recipe complexity (e.g., quick and easy, gourmet elaborate)")
    diversity_openness: str = Field(description="Willingness to try new cuisines (e.g., adventurous, selective, traditionalist, not defined)")

    notes: str = Field(description="Brief summary explaining the users overall food personality and any notable patterns in their preferences")
    justification: str = Field(description="Detailed explanation of how the profile was determined based on the users interaction history and ratings. Describe why the liked cuisines, cuisine preference, dietary preference, food preferences, cuisine preferences, dietary preferences, flavor preferences, daypart preferences, and lifestyle tags were chosen. Is not allowed to use quotes or complex punctuation in this field. Keep it between 100 and 200 words not more.")
    user_story: str = Field(description="Predictive narrative about the user s culinary evolution and potential future preferences. Describes their food journey, emerging patterns, and likely directions for taste exploration. Written to help predict what they might enjoy next based on their current trajectory and evolving palate.")
    future_preferences: str = Field(description="Speculative insights into the types of recipes and cuisines the user may be inclined to explore in the future. Based on their current preferences, suggest new food categories, cooking styles, or dietary trends they might be open to trying next. This helps in anticipating their evolving culinary interests.")


In [ ]:
user_profile_prompt = f"""Generate a structured user profile that captures their culinary tastes, dietary preferences, flavor inclinations, among others. This user profile will be used then for a Recommendation System. Ensure the profile is concise, reasonable and accurately reflects the users food personality based on their interaction history. Please provide a structured profile of the user using the following format: {schema_to_prompt_with_descriptions(UserProfile)}. Each fill of the structured output doesnt need to take more than 200 words keep it in mind. IMPORTANT: Do not use quotation marks or complex punctuation in your response. Use simple words and avoid any quotes, apostrophes, or special characters. Use the following interaction history as reference:"""

user_profile_generation_query = f"""
WITH ai_responses AS (
  SELECT 
    s.user_id, 
    s.n_history,
    s.history_string,
    AI.GENERATE(('{user_profile_prompt}', s.history_string),
        connection_id => '{CONNECTION_ID}',
        endpoint => 'gemini-2.5-flash',
        model_params => JSON '{{"generationConfig":{{"temperature": 1.0, "maxOutputTokens": 2048, "thinking_config": {{"thinking_budget": 1024}} }} }}',
        output_schema => 'liked_cuisines ARRAY<STRING>, cuisine_preference STRING, dietary_preference STRING, food_preferences ARRAY<STRING>, top_cuisine_choices ARRAY<STRING>, dietary_preferences ARRAY<STRING>, flavor_preferences ARRAY<STRING>, daypart_preferences ARRAY<STRING>, lifestyle_tags ARRAY<STRING>, convenience_preference STRING, diversity_openness STRING, notes STRING, justification STRING, user_story STRING, future_preferences STRING'
    ) AS ai_result
  FROM (SELECT * FROM `{USERS_PARSED}`) s
)
SELECT 
  *,
  ai_result.full_response AS user_profile,
  JSON_EXTRACT_SCALAR(ai_result.full_response, '$.candidates[0].content.parts[0].text') AS user_profile_text
FROM ai_responses
"""

# Vector Search vs ALS


We begin creating the text embeddings for both users and recipes, as follow:

In [ ]:
client.query_and_wait(f"""
ALTER TABLE `{PROJECT_ID}.{USERS_PROFILES_TABLE}`
ADD COLUMN text_embedding ARRAY<FLOAT64>
""")

client.query_and_wait(f"""
UPDATE `{PROJECT_ID}.{USERS_PROFILES_TABLE}` AS t
SET t.text_embedding = s.ml_generate_embedding_result
FROM (
  SELECT
    user_id,
    ml_generate_embedding_result
  FROM
    ML.GENERATE_EMBEDDING(
      MODEL `{SCHEMA_NAME}.text_embedding_model`,
      (
        SELECT
          user_id,
          user_profile_text AS content
        FROM `{PROJECT_ID}.{USERS_PROFILES_TABLE}`
      ),
      STRUCT(TRUE AS flatten_json_output, {OUT_DIM} AS OUTPUT_DIMENSIONALITY, 'RETRIEVAL_QUERY' AS task_type)
    )
) AS s
WHERE t.user_id = s.user_id
""")

We could then reduce the search space of the vector search query using business logic. For example, we could filter recipes by cuisine, meal type, or dietary restrictions based on the user's preferences. This would help in retrieving more relevant recommendations. But, it is out of the scope of this notebook.

## Naive Approach (Collaborative Filtering)

## Simple Vector Search 

## HyDE (Hypothetical Document Embedding) Approach

## ALS

We see in other competitions like [H&M](https://www.kaggle.com/code/julian3833/h-m-implicit-als-model-0-014) and in papers like ...

If we'd had a more reliable retriever (powered by any algorithm, heuristic or methodology), we could have used `AI.GENERATE` method to re-rank the retrieved candidates as in [LlamaRec](https://arxiv.org/pdf/2311.02089). Some RecSys ideas can be found in more advanced papers like [LRU](https://arxiv.org/pdf/2310.02367)

# Hypothesis (Explanation) Generation

Inspired by the work of [Spotify](https://arxiv.org/abs/2508.08777)

# UI with Streamlit

# Summary and Conclusions

## Feedback on BigQuery AI features
- We found difficult at the beggining to pass the description of the attributes of the Pydantic models (used in libraries like LangChain and Instructor) as schema for the LLM calls. As you see in the code, we had to create a function to create a part of the prompt with the description of each attribute. It would be great to have a more straightforward way to do this.
- The LLM calls are quite slow, even using the lightweight models. We had to limit the number of recipes and users to process. It would be great to have a way to speed up the calls, and show the progress of the calls.
- If a response is too lengthy and surpasses the maximum tokens limit, it may cause errors related to quotation marks. Improving error messages and handling for these cases would make debugging easier and prevent the entire query from failing due to such edge cases.
- We found that sometimes the Gemini Embedding 001 model returns empty embeddings for certain inputs. ...

## User Survey on BigQuery AI features